In [1]:

import sys
import os
sys.path.append(os.path.abspath(".."))

from pypdf import PdfReader
from utils.database_utils import insert_sections_into_db, create_table_if_not_exists

In [2]:
reader = PdfReader('pdfs/Dress Code Policy.pdf')

In [3]:
print(len(reader.pages))

6


In [4]:
page = reader.pages[0]
text = page.extract_text()
print(text)

ELSEWEDY 
ELECTRIC S —— 
ELSEWEDY ELECTRIC GROUP 
Dress Code & Personal Appearance Policy 
www.elsewedyelectric.com


In [5]:
pages_text = {}

for i in range(len(reader.pages)): 
    text = reader.pages[i].extract_text()
    pages_text.update({f"Page_{i + 1}" : text})
    

In [6]:
pages_text["Page_3"]

'ELSEWEDY \nELECTRIC ~— \n1. Purpose \nThis policy is designed fo guide employees on the required standards of dress and \nappearance. \nEmployees must maintain an appropriate standard of dress and personal appearance at \nwork and conduct themselves in a professional manner at all times both within the \nworkplace and when representing the Company. \nThis Policy is a minimum standard; where local legislations define higher standards; the \nGroup shall comply with them. \n2. Applicability \nThis policy applies to all the operating companies and subsidiaries directly or indirectly \ncontrolled by Elsewedy Electric, and all the geographical regions where Elsewedy Electric \ncompanies and subsidiaries are operating. \n3. Definitions \nBusiness Formal Dress Code includes suits and ties for men, while women require business \nsuits with pants or a long skirt and a jacket. \nSemi-Formal Dress Code includes suits, jackets, shirts, skirts and pants that are typical of \nformal business attire 

In [7]:
import re

def split_numbered_sections(text):
    # This matches headers like "1. Purpose", "2. Applicability", etc.
    pattern = r'(?P<full>(?P<number>\d+)\.\s+(?P<title>[A-Za-z ]{3,50}))'

    matches = list(re.finditer(pattern, text))
    sections = []

    for i, match in enumerate(matches):
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        section_number = int(match.group("number"))
        section_title = match.group("title").strip()
        section_text = text[start:end].strip()

        sections.append({
            "section_number": section_number,
            "title": section_title,
            "content": section_text
        })

    return sections


In [8]:
text = (
    pages_text["Page_3"]
    + "\n"
    + pages_text["Page_4"]
    + "\n"
    + pages_text["Page_5"]
    + "\n"
    + pages_text["Page_6"]
)

In [9]:
sections = split_numbered_sections(text)

In [10]:
# Step 3: Add metadata (source_file + page)
for i, section in enumerate(sections):
    section["source_file"] = "Dress Code Policy"
    section["page"] = 3 + i  # Adjust if needed

In [11]:
print(sections)

[{'section_number': 1, 'title': 'Purpose', 'content': 'This policy is designed fo guide employees on the required standards of dress and \nappearance. \nEmployees must maintain an appropriate standard of dress and personal appearance at \nwork and conduct themselves in a professional manner at all times both within the \nworkplace and when representing the Company. \nThis Policy is a minimum standard; where local legislations define higher standards; the \nGroup shall comply with them.', 'source_file': 'Dress Code Policy', 'page': 3}, {'section_number': 2, 'title': 'Applicability', 'content': 'This policy applies to all the operating companies and subsidiaries directly or indirectly \ncontrolled by Elsewedy Electric, and all the geographical regions where Elsewedy Electric \ncompanies and subsidiaries are operating.', 'source_file': 'Dress Code Policy', 'page': 4}, {'section_number': 3, 'title': 'Definitions', 'content': 'Business Formal Dress Code includes suits and ties for men, whil

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # small, fast, accurate

# Example: embed all your sections
texts = [f"{s['title']}\n{s['content']}" for s in sections]
embeddings = model.encode(texts, show_progress_bar=True)

c:\Users\Fady\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


In [13]:
create_table_if_not_exists()

insert_sections_into_db(sections, embeddings)

✅ All sections inserted into the database.


In [14]:
# import faiss
# import numpy as np

# # Step 1: convert to numpy array
# emb_matrix = np.array(embeddings).astype("float32")

# # Step 2: create index
# index = faiss.IndexFlatL2(emb_matrix.shape[1])  # L2 = Euclidean distance
# index.add(emb_matrix)  # Add all embeddings

# # Optional: store metadata for later lookup
# metadata = sections  # Keep same order


In [15]:
# faiss.write_index(index, "embeddings/dress_code_policy.faiss")


In [16]:
# import json
# with open("embeddings/metadata.json", "w") as f:
#     json.dump(metadata, f)


In [17]:
# query_embedding = model.encode(["When can we wear casual clothes?"])[0]

In [18]:
# D, I = index.search(np.array([query_embedding]).astype("float32"), k=3)

# # I contains indices of most relevant sections
# relevant_chunks = [metadata[i] for i in I[0]]


In [19]:
# print(relevant_chunks)